In [1]:
import pandas as pd
from _2_get_pysent_scores import *

In [29]:
# Import libraries
import torch
from transformers import AutoModel, AutoTokenizer, pipeline, AutoModelForSequenceClassification
from TweetNormalizer import normalizeTweet
import pandas as pd
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
from pysentimiento_master.pysentimiento.preprocessing import preprocess_tweet
from pysentimiento_master.pysentimiento.preprocessing import *
from tqdm import tqdm
from scipy.special import softmax
import urllib.request
import csv

In [3]:
# List of teams and years to iterate through for file names
team_search_queries = {'giants': ["2012"],
                      'devils': ["2012"],
                      'knicks': ["2012"],
                      'mets': ["2012"],
                      'yankees': ["2012"]}

In [9]:
# Outputs pre-processed data - DON'T RUN UNLESS RE-preprocessing data
for team in team_search_queries.keys():
    
    year = 2012
    # Access the correct year/team file
    file_dir = r"C:\Users\clark\OneDrive - University of Warwick\Diss\1. Twitter API\Data\Data - Scraped Tweets\FINAL Tweets"
    file = "\{}_{}.csv".format(team,year)

    file_path = file_dir + file

    unclean_data = pd.read_csv(file_path, index_col = "index")
    unclean_data = unclean_data.drop("Unnamed: 0", axis=1)
    
    # Pre-process tweets
    clean_data = pysent_clean_tweet(unclean_data)
    
    # Save the file down in:
    save_dir = r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\Preprocessed tweets"
    file_name = "\{}_{}_processed.csv".format(year,team)
    clean_data.to_csv(save_dir + file_name)

# All the below is working notes on how to get a mood tranformer working!
## Try transformers

In [2]:
# Load in a pre-processed file
knicks = pd.read_csv(r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\Preprocessed tweets\2012_knicks_processed.csv", index_col = 0)

In [13]:
# look at knicks
knicks

,author_id,tweet_id,text,created_at
index,,,,
0,209771195.0,1.536225e+17,@USER lmao !! Fuck it it's basketball season n...,2012-01-01 23:43:53+00:00
1,40612777.0,1.536038e+17,Just won my first online match in nba2 k12 nyk...,2012-01-01 22:29:34+00:00
2,30281791.0,1.535922e+17,So No One gonna recognize that Josh Harrelson ...,2012-01-01 21:43:37+00:00
3,49532208.0,1.535882e+17,@USER @USER @USER I write about knicks & we R ...,2012-01-01 21:27:44+00:00
4,49532208.0,1.535850e+17,@USER How about following me back? I write abo...,2012-01-01 21:14:46+00:00
...,...,...,...,...
2870,34889095.0,2.857623e+17,"Since 1996, the overly bashed,but of all nba j...",2012-12-31 15:00:11+00:00
2871,374801988.0,2.856106e+17,Time for me to let football go @USER @USER cow...,2012-12-31 04:57:32+00:00
2872,44649637.0,2.856062e+17,On another note and I can now put my energy in...,2012-12-31 04:39:56+00:00


In [11]:
# create the sentiment analyser pipeline
#model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-sentiment-analysis")
#tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

# Tryy wit "text-classification"
classifier = pipeline("text-classification", model = model, tokenizer = tokenizer)

Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [8]:
test_tweet = knicks["text"].iloc[1]
test_tweet

'Just won my first online match in nba2 k12 nyknicks over the Heat.. @USER dropped 26...im 1-0...now ill retire! Perfect record!'

In [12]:
test_output = classifier(test_tweet)

In [13]:
test_output

[{'label': 'optimism', 'score': 0.87562495470047}]

In [22]:
# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [25]:
encoded_input = tokenizer(test_tweet, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Celebrating my promotion 😎"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) joy 0.8756
2) optimism 0.0655
3) anger 0.045
4) sadness 0.014


## pysent

In [27]:
#model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-sentiment-analysis")
#tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-sentiment-analysis")
# Tryy wit "text-classification"
classifier = pipeline("text-classification", model = model, tokenizer = tokenizer)

The model 'RobertaForSequenceClassification' is not supported for token-classification. Supported models are ['QDQBertForTokenClassification', 'FNetForTokenClassification', 'LayoutLMv2ForTokenClassification', 'RemBertForTokenClassification', 'CanineForTokenClassification', 'RoFormerForTokenClassification', 'BigBirdForTokenClassification', 'ConvBertForTokenClassification', 'LayoutLMForTokenClassification', 'DistilBertForTokenClassification', 'CamembertForTokenClassification', 'FlaubertForTokenClassification', 'XLMForTokenClassification', 'XLMRobertaForTokenClassification', 'LongformerForTokenClassification', 'RobertaForTokenClassification', 'SqueezeBertForTokenClassification', 'BertForTokenClassification', 'MegatronBertForTokenClassification', 'MobileBertForTokenClassification', 'XLNetForTokenClassification', 'AlbertForTokenClassification', 'ElectraForTokenClassification', 'FunnelForTokenClassification', 'MPNetForTokenClassification', 'DebertaForTokenClassification', 'DebertaV2ForTokenC

In [30]:
# from pysentimiento import create_analyzer
# analyzer = create_analyzer(task="emotion", lang="en")

ModuleNotFoundError: No module named 'pysentimiento.metrics'

## Roberta

In [31]:
model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
# Tryy wit "text-classification"
classifier = pipeline("text-classification", model = model, tokenizer = tokenizer, return_all_scores=True)


Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/294 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [39]:
test_output = classifier(test_tweet)
test_output[0][6]['score']

0.5532079339027405

In [40]:
# FUNCION THAT TAKES A DATAFRAME AS INPUT AND OUTPUTS IT WITH LABELS AND SCORES
def get_mood(df):
    
    model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
    tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
 
    classifier = pipeline("text-classification", model = model, tokenizer = tokenizer, return_all_scores=True)
    
    # Take a list of the tweets and find the sentiment and score of each 
    tweets = list(df["text"])
    
    labels = []
    scores = []
    
    for sent in tqdm(tweets):
        
        try:
            results = classifier(sent)
            #Assign label to label
            #df.at[i,"label"] = results[0][0]["label"]
            #df.at[i,"score"] = results[0][0]["score"]
            
            labels.append(results[0][6]["label"])
            scores.append(results[0][6]['score'])
        # Get the exception to print out the error and for which team/time then pass to next month
        except:
        # except Exception as e:
        #     print(e)
            labels.append("DELETE")
            scores.append(100000)
            print("The tweet was: ",sent)
            pass

    # Create an empty col for label and score
    df = df.assign(label = labels)
    df = df.assign(score = scores)
    
    return df

In [ ]:
test_mood = get_mood(knicks)

 40%|██████████████████████████████▋                                              | 9350/23482 [44:41<37:42,  6.25it/s]

The tweet was:  Steve Novak just makes my heart sing in a totally platonic way every time he hits a three and then does the Double Check. nyknicks


 40%|██████████████████████████████▊                                              | 9382/23482 [44:48<47:12,  4.98it/s]